In [2]:
# Only run on initial setup or restart of EC2 instance
%pip install tensorflow
%pip install opencv-python==4.3.0.36
%pip install imutils
%pip install google-colab
%pip install scikeras

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached google_auth-2.15.0-py2.py3-none-any.whl (177 kB)
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
  Using cached opencv_python-4.3.0.36-cp37-cp37m-manylinux2014_x86_64.whl (43.7 MB)
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
  Using cached imutils-0.5.4-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached tornado-4.5.3-cp37-cp37m-linux_x86_64.whl
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached requests-2.

In [3]:
from tensorflow import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard,  ModelCheckpoint
from keras.layers import LeakyReLU
from sklearn import metrics
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
epochs = 10                 # Number of Training Epochs
num_classes = 10            # Number of classes in the Fashion MNIST dataset
batch_size = 256
img_rows, img_cols = 28, 28 # Pixel sizes of the Images in the Dataset

In [5]:
# Get the data from the keras repository
mnist_data = fashion_mnist.load_data()

# x = images, y = labels
x = mnist_data[0][0]
y = mnist_data[0][1]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one-hot encoded)
oh_y = keras.utils.to_categorical(encoded_Y)

# Process the date into the right tensor shape.
if K.image_data_format() == 'channels first':
    x = x.reshape(x.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x = x.reshape(x.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# Cast to a 32 bit float and then scale so the value is a float normalized between 0 and 1. 
x = x.astype('float32')
x /= 255

# Function to decode one-hot encoding later on when evaluating performance.
def decode_one_hot(y):
    y_classes = [np.argmax(yi, axis=None, out=None) for yi in y]
    return y_classes

# Topological skeleton preprocessing function to be used with the Image Augmentation Data Generator.
def topological_skel(image):
    img = image
    size = np.size(img)
    skel = np.zeros(img.shape,np.uint8)
    ret,img = cv2.threshold(img,127,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False
    while( not done):
        eroded = cv2.erode(img,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(img,temp)
        skel = cv2.bitwise_or(skel,temp)
        img = eroded.copy()
        zeros = size - cv2.countNonZero(img)
        if zeros==size:
            done = True
    return skel

In [6]:
def baseline_model():
    # Define sequential model and add layers
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation=keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape)) 
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    # Print model summary
    model.summary()
    my_callbacks = [ModelCheckpoint('model_out.hdf5', monitor='acc', save_best_only=True, period=1)]
    # Compile the model so we can fit it.
    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=keras.optimizers.Nadam(), 
                  metrics=['accuracy'])
    return model

In [7]:

estimator = KerasClassifier(model=baseline_model, epochs=20, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, x, oh_y, cv=kfold)

print('Results: ', results)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 32)          0

NameError: name 'model' is not defined

In [ ]:
epoch_list = list(range(1, len(hist.history['accuracy']) + 1))

# Plot training and test accuracy
plt.plot(epoch_list, hist.history['accuracy'], epoch_list, hist.history['val_accuracy'])
plt.legend(("Training Accuracy", "Validation Accuracy"))
plt.show()

# Plot training and test loss
plt.plot(epoch_list, hist.history['loss'], epoch_list, hist.history['val_loss'])
plt.legend(("Training Loss", "Validation Loss"))
plt.show()

In [19]:
import cv2
from imutils import build_montages
from google.colab.patches import cv2_imshow

# Initialize list of output images
images = []

# Define labels in correct order
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]
 
# Randomly select testing fashion items
for i in np.random.choice(np.arange(0, len(y)), size=(16,)):
    # Classify the clothing
    probs = model.predict(x_test[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    label = labelNames[prediction[0]]

    # Extract the image from the testData if using "channels_first" ordering
    if K.image_data_format() == "channels_first":
        image = (x[i][0] * 255).astype("uint8")

    # Else use "channels_last" ordering
    else:
        image = (x[i] * 255).astype("uint8")

    # Initialize the text label color as green (correct)
    color = (0, 255, 0)

    # Otherwise, the class label prediction is red (incorrect) 
    if prediction[0] != np.argmax(y_test[i]):
        color = (0, 0, 255)

    # Merge the channels into one image and resize the image from 28x28 to 96x96
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    # Draw the predicted label on the image
    image = cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.60, color, 2)

    # Add the image to list of output images
    images.append(image)
# Construct the montage for the images
montage = build_montages(images, (96, 96), (4, 4))[0]
 
# Show the output montage
cv2_imshow(montage)

NameError: name 'model' is not defined